In [1]:
import argparse
import json
import random
import os
import os.path as osp
import shutil
from icecream import ic
import pathlib
import pickle

import numpy as np
import torch
from mmcv import Config, mkdir_or_exist
from tqdm import tqdm  # Progress bar

import wandb
# Functions to calculate metrics and show the relevant chart colorbar.
from functions import compute_metrics, save_best_model, load_model, slide_inference, batched_slide_inference
# Custom dataloaders for regular training and validation.
from loaders import (AI4ArcticChallengeDataset, AI4ArcticChallengeTestDataset,
                     get_variable_options)
#  get_variable_options
from unet import UNet  # Convolutional Neural Network model
from swin_transformer import SwinTransformer  # Swin Transformer
# -- Built-in modules -- #
from utils import colour_str

from test_upload_function import test
import segmentation_models_pytorch as smp

def create_train_and_validation_scene_list(train_options):
    '''
    Creates the a train and validation scene list. Adds these two list to the config file train_options

    '''
    with open(train_options['path_to_env'] + train_options['train_list_path']) as file:
        train_options['train_list'] = json.loads(file.read())

    # Convert the original scene names to the preprocessed names.
    train_options['train_list'] = [file[17:32] + '_' + file[77:80] +
                                   '_prep.nc' for file in train_options['train_list']]

    # # Select a random number of validation scenes with the same seed. Feel free to change the seed.et
    # # np.random.seed(0)
    # train_options['validate_list'] = np.random.choice(np.array(
    #     train_options['train_list']), size=train_options['num_val_scenes'], replace=False)

    # load validation list
    with open(train_options['path_to_env'] + train_options['val_path']) as file:
        train_options['validate_list'] = json.loads(file.read())
    # Convert the original scene names to the preprocessed names.
    train_options['validate_list'] = [file[17:32] + '_' + file[77:80] +
                                      '_prep.nc' for file in train_options['validate_list']]

    # from icecream import ic
    # ic(train_options['validate_list'])
    # Remove the validation scenes from the train list.
    train_options['train_list'] = [scene for scene in train_options['train_list']
                                   if scene not in train_options['validate_list']]
    print('Options initialised')


def create_dataloaders(train_options):
    '''
    Create train and validation dataloader based on the train and validation list inside train_options.

    '''
    # Custom dataset and dataloader.
    dataset = AI4ArcticChallengeDataset(
        files=train_options['train_list'], options=train_options, do_transform=True)

    dataloader_train = torch.utils.data.DataLoader(
        dataset, batch_size=None, shuffle=True, num_workers=train_options['num_workers'], pin_memory=True)
    # - Setup of the validation dataset/dataloader. The same is used for model testing in 'test_upload.ipynb'.

    dataset_val = AI4ArcticChallengeTestDataset(
        options=train_options, files=train_options['validate_list'], mode='train_val')

    dataloader_val = torch.utils.data.DataLoader(
        dataset_val, batch_size=None, num_workers=train_options['num_workers_val'], shuffle=False)

    return dataloader_train, dataloader_val


/home/fernando/anaconda3/envs/AI4Artic2/lib/python3.9/site-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(


In [2]:

config_path = 'configs/edge_consitency/get_output.py'

checkpoint_path = 'checkpoints/best_model_ds_sgd_re_domain_adp.pth'

ic(config_path)
cfg = Config.fromfile(config_path)
train_options = cfg.train_options
# Get options for variables, amsrenv grid, cropping and upsampling.
train_options = get_variable_options(train_options)


if torch.cuda.is_available():
    print(colour_str('GPU available!', 'green'))
    print('Total number of available devices: ',
            colour_str(torch.cuda.device_count(), 'orange'))
    device = torch.device(f"cuda:{train_options['gpu_id']}")
else:
    print(colour_str('GPU not available.', 'red'))
    device = torch.device('cpu')
    print('GPU setup completed!')


if train_options['model_selection'] == 'unet':
    net = UNet(options=train_options).to(device)
elif train_options['model_selection'] == 'swin':
    net = SwinTransformer(options=train_options).to(device)
elif train_options['model_selection'] == 'h_unet':
    from unet import H_UNet
    net = H_UNet(options=train_options).to(device)
elif train_options['model_selection'] == 'h_unet_argmax':
    from unet import H_UNet_argmax
    net = H_UNet_argmax(options=train_options).to(device)    
else:
    raise 'Unknown model selected'

print(f"\033[91m Finetune model from {checkpoint_path}\033[0m")
_ = load_model(net, checkpoint_path)



create_train_and_validation_scene_list(train_options)

dataloader_train, dataloader_val = create_dataloaders(train_options)


ic| config_path: 'configs/edge_consitency/get_output.py'


GPU available!
Total number of available devices:  1
 Finetune model from checkpoints/best_model_ds_sgd_re_domain_adp.pth
Options initialised


100%|██████████| 11/11 [00:14<00:00,  1.33s/it]


In [11]:
val_iter =iter(dataloader_val)
net.eval() 

outputs = []
for i in range(10):
    batch = next(val_iter)
    with torch.no_grad(), torch.cuda.amp.autocast():
        output = net(batch[0].to(device, non_blocking=True))
        for chart in train_options['charts']:
            output[chart].to('cpu').detach()
    outputs.append(output)
    

In [ ]:
# save pickle
with open('model_outputs', 'wb') as model_outputs_file:
 
  # Step 3
  pickle.dump(outputs, model_outputs_file)
